创建history对象

In [1]:
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { HumanMessage, AIMessage } from "@langchain/core/messages";
var history = new ChatMessageHistory();
//存储消息
await history.addMessage(new HumanMessage('hi'))
await history.addMessage(new AIMessage('What can I do for you?'))
//获取消息
var messages = await history.getMessages()
console.log(messages)

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "hi", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "hi",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "What can I do for you?",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "What can I do for you?",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]


手动维护 chat history

In [2]:
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { HumanMessage, AIMessage } from "@langchain/core/messages";
import {load} from 'dotenv'
const env = await load()
const process = {env}
var chatModel = new ChatOpenAI({
  configuration: {
    baseURL: process.env.baseURL,
  },
});
//MessagesPlaceholder 就是创建一个名为 history_message 的插槽，chain 中对应的参数将会替换这部分
var prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    `You are a helpful assistant. Answer all questions to the best of your ability.     You are talkative and provides lots of specific details from its context. 
    If the you does not know the answer to a question, it truthfully says you do not know.`,
  ],
  new MessagesPlaceholder("history_message"),
]);
var chain = prompt.pipe(chatModel);

//然后，我们创建一个 chat history，然后向其中添加一条历史记录，并请求 ai 回答

var history = new ChatMessageHistory();
await history.addMessage(new HumanMessage("你好，我是赵恒盛"));
var res1 = await chain.invoke({
  history_message: await history.getMessages(),
});
//res1是 llm 返回的消息，所以也应该添加到 chat history 中，让我们添加进去，并且添加一条人类的新提问
console.log(res1);
await history.addMessage(res1);
await history.addMessage(new HumanMessage('我是谁？'))
var res2 = await chain.invoke({
  history_message: await history.getMessages(),
});
console.log(res2);


AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "您好，赵恒盛先生。有什么我可以帮您的吗？",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "您好，赵恒盛先生。有什么我可以帮您的吗？",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 25, promptTokens: 78, totalTokens: 103 },
    finish_reason: "stop"
  },
  tool_calls: [],
  invalid_tool_calls: []
}
AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "我是赵恒盛先生的个人助理AI，我被设计成能帮助他处理各种各样的问题。我可以帮助他管理日程安排、发送消息、回答问题，甚至是帮助他规划旅行路线。作为AI，我会尽我所能地提供有用和准确的信息，让赵恒盛先生能"... 39 more characters,
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  con

自动维护chat history

In [2]:
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";
import { ChatOpenAI } from "@langchain/openai";
import { ChatMessageHistory } from "langchain/stores/message/in_memory";
import { RunnableWithMessageHistory } from "@langchain/core/runnables";
import { HumanMessage, AIMessage } from "@langchain/core/messages";
import { load } from "dotenv";
const env = await load();
const process = { env };
var chatModel = new ChatOpenAI({
  configuration: {
    baseURL: process.env.baseURL,
  },
});
var prompt = ChatPromptTemplate.fromMessages([["system", "你是一个乐于助人的助手。尽你所能回答所有问题。"], new MessagesPlaceholder("history_message"), ["human", "{input}"]]);
var history = new ChatMessageHistory();
var chain = prompt.pipe(chatModel);
var chainWithHistory = new RunnableWithMessageHistory({
  runnable: chain,
  getMessageHistory: (_sessionId) => history,
  inputMessagesKey: "input",
  historyMessagesKey: "history_message",
});
var res1 = await chainWithHistory.invoke(
  {
    input: "你好，我是赵恒盛",
  },
  { configurable: { sessionId: "none" } }
);
console.log(res1);
var res2 = await chainWithHistory.invoke(
  {
    input: "我的名字叫什么？",
  },
  { configurable: { sessionId: "none" } }
);
console.log(res2);


AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "你好，赵恒盛先生，有什么可以帮到您的吗？",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "你好，赵恒盛先生，有什么可以帮到您的吗？",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 25, promptTokens: 47, totalTokens: 72 },
    finish_reason: "stop"
  },
  tool_calls: [],
  invalid_tool_calls: []
}
AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "您告诉我您的名字是赵恒盛。",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "您告诉我您的名字是赵恒盛。",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefin